<a href="https://colab.research.google.com/github/mertege/time_series_classification/blob/main/3_Spectrograms_CI4R_Datasets_77GHz_Multi_Class_LSTM_with_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Burada CI4R dataseti ile Multi Class Classification yapıldı.
# Sadece 77GHz train ve test edildi.
# Bidirectional(GRU) ve Bidirectional(LSTM) ile paralel olarak koşuyor ve ayrı ayrı attention yapılıyor. Sonrasında concat ile birleştiliyor. 
# Çözünürlük 216x216
# Mean test accuracy is 0.858, max test accuracy is 0.863,
# Min test accuracy is 0.855,  std of test accuracy is 0.003.
# Time elapsed through all process: 1799.72 sec

In [ ]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Conv1D,Activation, Dropout, Flatten, Dense, BatchNormalization, Normalization, Input, Conv2D, MaxPooling2D, Concatenate, GRU, LSTM, TimeDistributed, Bidirectional, ReLU
import tensorflow as tf
import scipy.io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import EarlyStopping
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K 
import gc
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 77 GHz
bending_77GHz                 = np.load('/content/drive/MyDrive/bending_77Ghz.npy')
bending_77GHz = np.float32(bending_77GHz)
label_bending_77GHz           = np.zeros((bending_77GHz.shape[0],1))
label_bending_77GHz[:]        = 1
crawling_77GHz                = np.load('/content/drive/MyDrive/crawling_77Ghz.npy')
crawling_77GHz = np.float32(crawling_77GHz)
label_crawling_77GHz          = np.zeros((crawling_77GHz.shape[0],1))
label_crawling_77GHz[:]       = 2
kneeling_77GHz                = np.load('/content/drive/MyDrive/kneeling_77Ghz.npy')
kneeling_77GHz = np.float32(kneeling_77GHz)
label_kneeling_77GHz          = np.zeros((kneeling_77GHz.shape[0],1))
label_kneeling_77GHz[:]       = 3
limping_with_RL_Stiff_77GHz   = np.load('/content/drive/MyDrive/limping_with_RL_Stiff_77Ghz.npy')
limping_with_RL_Stiff_77GHz = np.float32(limping_with_RL_Stiff_77GHz)
label_limping_with_RL_Stiff_77GHz           = np.zeros((limping_with_RL_Stiff_77GHz.shape[0],1))
label_limping_with_RL_Stiff_77GHz[:]        = 4
picking_up_an_object_77GHz    = np.load('/content/drive/MyDrive/picking_up_an_object_77Ghz.npy')
picking_up_an_object_77GHz = np.float32(picking_up_an_object_77GHz)
label_picking_up_an_object_77GHz          = np.zeros((picking_up_an_object_77GHz.shape[0],1))
label_picking_up_an_object_77GHz[:]       = 5
scissors_gait_77GHz           = np.load('/content/drive/MyDrive/scissors_gait_77Ghz.npy')
scissors_gait_77GHz = np.float32(scissors_gait_77GHz)
label_scissors_gait_77GHz          = np.zeros((scissors_gait_77GHz.shape[0],1))
label_scissors_gait_77GHz[:]       = 6
short_steps_77GHz             = np.load('/content/drive/MyDrive/short_steps_77Ghz.npy')
short_steps_77GHz = np.float32(short_steps_77GHz)
label_short_steps_77GHz          = np.zeros((short_steps_77GHz.shape[0],1))
label_short_steps_77GHz[:]       = 7
sitting_77GHz                 = np.load('/content/drive/MyDrive/sitting_77Ghz.npy')
sitting_77GHz = np.float32(sitting_77GHz)
label_sitting_77GHz          = np.zeros((sitting_77GHz.shape[0],1))
label_sitting_77GHz[:]       = 8
walking_away_from_Radar_77GHz = np.load('/content/drive/MyDrive/walking_away_from_Radar_77Ghz.npy')
walking_away_from_Radar_77GHz = np.float32(walking_away_from_Radar_77GHz)
label_walking_away_from_Radar_77GHz         = np.zeros((walking_away_from_Radar_77GHz.shape[0],1))
label_walking_away_from_Radar_77GHz[:]       = 9
Walking_on_both_toes_77GHz    = np.load('/content/drive/MyDrive/Walking_on_both_toes_77Ghz.npy')
Walking_on_both_toes_77GHz = np.float32(Walking_on_both_toes_77GHz)
label_Walking_on_both_toes_77GHz         = np.zeros((Walking_on_both_toes_77GHz.shape[0],1))
label_Walking_on_both_toes_77GHz[:]       = 10
Walking_towards_radar_77GHz   = np.load('/content/drive/MyDrive/Walking_towards_radar_77Ghz.npy')
Walking_towards_radar_77GHz = np.float32(Walking_towards_radar_77GHz)
label_Walking_towards_radar_77GHz          = np.zeros((Walking_towards_radar_77GHz.shape[0],1))
label_Walking_towards_radar_77GHz[:]       = 11
# # 24 GHz
# bending_24GHz                 = np.load('/content/drive/MyDrive/bending_24GHz.npy')
# label_bending_24GHz           = np.zeros((bending_24GHz.shape[0],1))
# label_bending_24GHz[:]        = 1
# crawling_24GHz                = np.load('/content/drive/MyDrive/crawling_24GHz.npy')
# label_crawling_24GHz          = np.zeros((crawling_24GHz.shape[0],1))
# label_crawling_24GHz[:]       = 2
# kneeling_24GHz                = np.load('/content/drive/MyDrive/kneeling_24GHz.npy')
# label_kneeling_24GHz          = np.zeros((kneeling_24GHz.shape[0],1))
# label_kneeling_24GHz[:]       = 3
# limping_with_RL_Stiff_24GHz   = np.load('/content/drive/MyDrive/limping with right stiff_24GHz.npy')
# label_limping_with_RL_Stiff_24GHz           = np.zeros((limping_with_RL_Stiff_24GHz.shape[0],1))
# label_limping_with_RL_Stiff_24GHz[:]        = 4
# picking_up_an_object_24GHz    = np.load('/content/drive/MyDrive/picking up object_24GHz.npy')
# label_picking_up_an_object_24GHz          = np.zeros((picking_up_an_object_24GHz.shape[0],1))
# label_picking_up_an_object_24GHz[:]       = 5
# scissors_gait_24GHz           = np.load('/content/drive/MyDrive/scissors gait_24GHz.npy')
# label_scissors_gait_24GHz          = np.zeros((scissors_gait_24GHz.shape[0],1))
# label_scissors_gait_24GHz[:]       = 6
# short_steps_24GHz             = np.load('/content/drive/MyDrive/short steps_24GHz.npy')
# label_short_steps_24GHz          = np.zeros((short_steps_24GHz.shape[0],1))
# label_short_steps_24GHz[:]       = 7
# sitting_24GHz                 = np.load('/content/drive/MyDrive/sitting_24GHz.npy')
# label_sitting_24GHz          = np.zeros((sitting_24GHz.shape[0],1))
# label_sitting_24GHz[:]       = 8
# walking_away_from_Radar_24GHz = np.load('/content/drive/MyDrive/walking away from radar_24GHz.npy')
# label_walking_away_from_Radar_24GHz         = np.zeros((walking_away_from_Radar_24GHz.shape[0],1))
# label_walking_away_from_Radar_24GHz[:]       = 9
# Walking_on_both_toes_24GHz    = np.load('/content/drive/MyDrive/walking on toes both_24GHz.npy')
# label_Walking_on_both_toes_24GHz         = np.zeros((Walking_on_both_toes_24GHz.shape[0],1))
# label_Walking_on_both_toes_24GHz[:]       = 10
# Walking_towards_radar_24GHz   = np.load('/content/drive/MyDrive/walking towards radar_24GHz.npy')
# label_Walking_towards_radar_24GHz          = np.zeros((Walking_towards_radar_24GHz.shape[0],1))
# label_Walking_towards_radar_24GHz[:]       = 11
# # Xethru
# bending_Xethru                 = np.load('/content/drive/MyDrive/bending_Xethru.npy')
# label_bending_Xethru           = np.zeros((bending_Xethru.shape[0],1))
# label_bending_Xethru[:]        = 1
# crawling_Xethru                = np.load('/content/drive/MyDrive/crawling_Xethru.npy')
# label_crawling_Xethru          = np.zeros((crawling_Xethru.shape[0],1))
# label_crawling_Xethru[:]       = 2
# kneeling_Xethru                = np.load('/content/drive/MyDrive/kneeling_Xethru.npy')
# label_kneeling_Xethru          = np.zeros((kneeling_Xethru.shape[0],1))
# label_kneeling_Xethru[:]       = 3
# limping_with_RL_Stiff_Xethru   = np.load('/content/drive/MyDrive/limping with right leg stiff_Xethru.npy')
# label_limping_with_RL_Stiff_Xethru           = np.zeros((limping_with_RL_Stiff_Xethru.shape[0],1))
# label_limping_with_RL_Stiff_Xethru[:]        = 4
# picking_up_an_object_Xethru    = np.load('/content/drive/MyDrive/ picking up an object_Xethru.npy')
# label_picking_up_an_object_Xethru          = np.zeros((picking_up_an_object_Xethru.shape[0],1))
# label_picking_up_an_object_Xethru[:]       = 5
# scissors_gait_Xethru           = np.load('/content/drive/MyDrive/scissor Gait_Xethru.npy')
# label_scissors_gait_Xethru          = np.zeros((scissors_gait_Xethru.shape[0],1))
# label_scissors_gait_Xethru[:]       = 6
# short_steps_Xethru             = np.load('/content/drive/MyDrive/short Step_Xethru.npy')
# label_short_steps_Xethru          = np.zeros((short_steps_Xethru.shape[0],1))
# label_short_steps_Xethru[:]       = 7
# sitting_Xethru                 = np.load('/content/drive/MyDrive/sitting_Xethru.npy')
# label_sitting_Xethru          = np.zeros((sitting_Xethru.shape[0],1))
# label_sitting_Xethru[:]       = 8
# walking_away_from_Radar_Xethru = np.load('/content/drive/MyDrive/Walking away from radar_Xethru.npy')
# label_walking_away_from_Radar_Xethru         = np.zeros((walking_away_from_Radar_Xethru.shape[0],1))
# label_walking_away_from_Radar_Xethru[:]       = 9
# Walking_on_both_toes_Xethru    = np.load('/content/drive/MyDrive/walking on both toes_Xethru.npy')
# label_Walking_on_both_toes_Xethru         = np.zeros((Walking_on_both_toes_Xethru.shape[0],1))
# label_Walking_on_both_toes_Xethru[:]       = 10
# Walking_towards_radar_Xethru   = np.load('/content/drive/MyDrive/Walking towards the radar_Xethru.npy')
# label_Walking_towards_radar_Xethru          = np.zeros((Walking_towards_radar_Xethru.shape[0],1))
# label_Walking_towards_radar_Xethru[:]       = 11

In [ ]:
label_concat = label_bending_77GHz
del label_bending_77GHz
label_concat = np.concatenate((label_concat,label_crawling_77GHz),axis=0)  
del label_crawling_77GHz
label_concat = np.concatenate((label_concat,label_kneeling_77GHz),axis=0)  
del label_kneeling_77GHz
label_concat = np.concatenate((label_concat,label_limping_with_RL_Stiff_77GHz),axis=0)  
del label_limping_with_RL_Stiff_77GHz
label_concat = np.concatenate((label_concat,label_picking_up_an_object_77GHz),axis=0)  
del label_picking_up_an_object_77GHz
label_concat = np.concatenate((label_concat,label_scissors_gait_77GHz),axis=0)  
del label_scissors_gait_77GHz
label_concat = np.concatenate((label_concat,label_short_steps_77GHz),axis=0)  
del label_short_steps_77GHz
label_concat = np.concatenate((label_concat,label_sitting_77GHz),axis=0)  
del label_sitting_77GHz
label_concat = np.concatenate((label_concat,label_walking_away_from_Radar_77GHz),axis=0)  
del label_walking_away_from_Radar_77GHz
label_concat = np.concatenate((label_concat,label_Walking_on_both_toes_77GHz),axis=0)  
del label_Walking_on_both_toes_77GHz
label_concat = np.concatenate((label_concat,label_Walking_towards_radar_77GHz),axis=0)  
del label_Walking_towards_radar_77GHz

# label_concat = label_bending_24GHz
# del label_bending_24GHz
# label_concat = np.concatenate((label_concat,label_crawling_24GHz),axis=0)  
# del label_crawling_24GHz
# label_concat = np.concatenate((label_concat,label_kneeling_24GHz),axis=0)  
# del label_kneeling_24GHz
# label_concat = np.concatenate((label_concat,label_limping_with_RL_Stiff_24GHz),axis=0)  
# del label_limping_with_RL_Stiff_24GHz
# label_concat = np.concatenate((label_concat,label_picking_up_an_object_24GHz),axis=0)  
# del label_picking_up_an_object_24GHz
# label_concat = np.concatenate((label_concat,label_scissors_gait_24GHz),axis=0)  
# del label_scissors_gait_24GHz
# label_concat = np.concatenate((label_concat,label_short_steps_24GHz),axis=0)  
# del label_short_steps_24GHz
# label_concat = np.concatenate((label_concat,label_sitting_24GHz),axis=0)  
# del label_sitting_24GHz
# label_concat = np.concatenate((label_concat,label_walking_away_from_Radar_24GHz),axis=0)  
# del label_walking_away_from_Radar_24GHz
# label_concat = np.concatenate((label_concat,label_Walking_on_both_toes_24GHz),axis=0)  
# del label_Walking_on_both_toes_24GHz
# label_concat = np.concatenate((label_concat,label_Walking_towards_radar_24GHz),axis=0)  
# del label_Walking_towards_radar_24GHz


# label_concat = label_bending_Xethru
# del label_bending_Xethru
# label_concat = np.concatenate((label_concat,label_crawling_Xethru),axis=0)  
# del label_crawling_Xethru
# label_concat = np.concatenate((label_concat,label_kneeling_Xethru),axis=0)  
# del label_kneeling_Xethru
# label_concat = np.concatenate((label_concat,label_limping_with_RL_Stiff_Xethru),axis=0)  
# del label_limping_with_RL_Stiff_Xethru
# label_concat = np.concatenate((label_concat,label_picking_up_an_object_Xethru),axis=0)  
# del label_picking_up_an_object_Xethru
# label_concat = np.concatenate((label_concat,label_scissors_gait_Xethru),axis=0)  
# del label_scissors_gait_Xethru
# label_concat = np.concatenate((label_concat,label_short_steps_Xethru),axis=0)  
# del label_short_steps_Xethru
# label_concat = np.concatenate((label_concat,label_sitting_Xethru),axis=0)  
# del label_sitting_Xethru
# label_concat = np.concatenate((label_concat,label_walking_away_from_Radar_Xethru),axis=0)  
# del label_walking_away_from_Radar_Xethru
# label_concat = np.concatenate((label_concat,label_Walking_on_both_toes_Xethru),axis=0)  
# del label_Walking_on_both_toes_Xethru
# label_concat = np.concatenate((label_concat,label_Walking_towards_radar_Xethru),axis=0)  
# del label_Walking_towards_radar_Xethru

In [ ]:
# 77 GHz
spectrogram_77GHz_concat = np.concatenate((bending_77GHz,crawling_77GHz),axis=0)
del bending_77GHz
del crawling_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,kneeling_77GHz               ),axis=0)
del kneeling_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,limping_with_RL_Stiff_77GHz  ),axis=0)
del limping_with_RL_Stiff_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,picking_up_an_object_77GHz   ),axis=0)
del picking_up_an_object_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,scissors_gait_77GHz          ),axis=0)
del scissors_gait_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,short_steps_77GHz            ),axis=0)
del short_steps_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,sitting_77GHz                ),axis=0)
del sitting_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,walking_away_from_Radar_77GHz),axis=0)
del walking_away_from_Radar_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,Walking_on_both_toes_77GHz   ),axis=0)
del Walking_on_both_toes_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,Walking_towards_radar_77GHz  ),axis=0)
del Walking_towards_radar_77GHz
# # 24 GHz
# spectrogram_24GHz_concat = np.concatenate((bending_24GHz,crawling_24GHz),axis=0)
# del bending_24GHz
# del crawling_24GHz
# spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,kneeling_24GHz               ),axis=0)
# del kneeling_24GHz
# spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,limping_with_RL_Stiff_24GHz  ),axis=0)
# del limping_with_RL_Stiff_24GHz
# spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,picking_up_an_object_24GHz   ),axis=0)
# del picking_up_an_object_24GHz
# spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,scissors_gait_24GHz          ),axis=0)
# del scissors_gait_24GHz
# spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,short_steps_24GHz            ),axis=0)
# del short_steps_24GHz
# spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,sitting_24GHz                ),axis=0)
# del sitting_24GHz
# spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,walking_away_from_Radar_24GHz),axis=0)
# del walking_away_from_Radar_24GHz
# spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,Walking_on_both_toes_24GHz   ),axis=0)
# del Walking_on_both_toes_24GHz
# spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,Walking_towards_radar_24GHz  ),axis=0)
# del Walking_towards_radar_24GHz


# # Xethru
# spectrogram_Xethru_concat = np.concatenate((bending_Xethru,crawling_Xethru),axis=0)
# del bending_Xethru
# del crawling_Xethru
# spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,kneeling_Xethru               ),axis=0)
# del kneeling_Xethru
# spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,limping_with_RL_Stiff_Xethru  ),axis=0)
# del limping_with_RL_Stiff_Xethru
# spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,picking_up_an_object_Xethru   ),axis=0)
# del picking_up_an_object_Xethru
# spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,scissors_gait_Xethru          ),axis=0)
# del scissors_gait_Xethru
# spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,short_steps_Xethru            ),axis=0)
# del short_steps_Xethru
# spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,sitting_Xethru                ),axis=0)
# del sitting_Xethru
# spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,walking_away_from_Radar_Xethru),axis=0)
# del walking_away_from_Radar_Xethru
# spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,Walking_on_both_toes_Xethru   ),axis=0)
# del Walking_on_both_toes_Xethru
# spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,Walking_towards_radar_Xethru  ),axis=0)
# del Walking_towards_radar_Xethru


In [ ]:
# One-Hot Encode Label
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(label_concat)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded_labels = onehot_encoder.fit_transform(integer_encoded)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# Concat range-doppler data
range_doppler_concat = spectrogram_77GHz_concat
range_doppler_concat_label = label_concat
# Shuffle concat range doppler
shuffle_indx = random.sample(range(0, range_doppler_concat.shape[0]), range_doppler_concat.shape[0]) # split validation data
range_doppler_concat_shuffle = range_doppler_concat[shuffle_indx,:,:,:]
range_doppler_concat_label_shuffle = range_doppler_concat_label[shuffle_indx,:]
# Concat range-doppler data
# spectrogram_concat = spectrogram_77GHz_concat
spectrogram_concat_label = onehot_encoded_labels
# Shuffle concat range doppler
# spectrogram_concat_shuffle = spectrogram_concat[shuffle_indx,:,:,:]
spectrogram_concat_label_shuffle = spectrogram_concat_label[shuffle_indx,:]
# Third Spectrogram
# third_spectrogram_concat = spectrogram_77GHz_concat
# third_spectrogram_concat_shuffle = third_spectrogram_concat[shuffle_indx,:,:,:]
del range_doppler_concat
del spectrogram_77GHz_concat

In [ ]:
# range_doppler_concat_shuffle_new = np.zeros((735,201,368,1))
# for ii in range(range_doppler_concat_shuffle.shape[0]):
#   aa = range_doppler_concat_shuffle[ii,:,:,0]
#   range_doppler_concat_shuffle_new[ii,:,:,0] = cv2.resize(aa, (368, 201),interpolation = cv2.INTER_CUBIC)
# range_doppler_concat_shuffle = range_doppler_concat_shuffle_new
# del aa
# del range_doppler_concat_shuffle_new

In [ ]:
# ---------------- Augmente and shuffle (train and test) data data ----------------
def mixup_augmentation(range_doppler_training_data, labels, repeat_of_mixup, alpha, beta):
    batch_size = range_doppler_training_data.shape[0]
    concat_images_range_doppler = np.zeros((batch_size*(repeat_of_mixup+1),range_doppler_training_data.shape[1],range_doppler_training_data.shape[2],range_doppler_training_data.shape[3]))
    concat_label = np.zeros((batch_size*(repeat_of_mixup+1),labels.shape[1]))

    # integer_training_labels = np.argmax(labels, axis=1)
    # np.where(integer_training_labels == 0)

    if repeat_of_mixup == 0:
      concat_images_range_doppler = range_doppler_training_data
      concat_label = labels      
    else:
      for ii in range(repeat_of_mixup):
        # shuffle train dataset
        shuffle_indx_1 = random.sample(range(0, range_doppler_training_data.shape[0]), range_doppler_training_data.shape[0]) # split validation data
        range_doppler_training_data_shuffled_1 = range_doppler_training_data[shuffle_indx_1,:,:,:]
        labels_shuffled_1 = labels[shuffle_indx_1,:]

        shuffle_indx_2 = random.sample(range(0, range_doppler_training_data.shape[0]), range_doppler_training_data.shape[0]) # split validation data
        range_doppler_training_data_shuffled_2 = range_doppler_training_data[shuffle_indx_2,:,:,:]
        labels_shuffled_2 = labels[shuffle_indx_2,:]

        # shuffle_indx_3 = random.sample(range(0, range_doppler_training_data.shape[0]), range_doppler_training_data.shape[0]) # split validation data
        # range_doppler_training_data_shuffled_3 = range_doppler_training_data[shuffle_indx_3,:,:,:]
        # labels_shuffled_3 = labels[shuffle_indx_3,:]

        # shuffle_indx_4 = random.sample(range(0, range_doppler_training_data.shape[0]), range_doppler_training_data.shape[0]) # split validation data
        # range_doppler_training_data_shuffled_4 = range_doppler_training_data[shuffle_indx_4,:,:,:]
        # labels_shuffled_4 = labels[shuffle_indx_4,:]

        # Sample lambda and reshape it to do the mixup
        # ll = np.random.beta(alpha, beta, (batch_size,1,1,1))
        gaussian_mean = 0.2
        gaussian_std = 0.02
        ll = np.random.normal(gaussian_mean, gaussian_std, (batch_size,1,1,1))
        x_l = np.reshape(ll, (batch_size,1,1,1))
        y_l = np.reshape(ll, (batch_size,1))
      
        ll_2 = np.random.normal(gaussian_mean, gaussian_std, (batch_size,1,1,1))
        x_l_2 = np.reshape(ll_2, (batch_size,1,1,1))
        y_l_2 = np.reshape(ll_2, (batch_size,1))

        ll_3 = np.random.normal(gaussian_mean, gaussian_std, (batch_size,1,1,1))
        x_l_3 = np.reshape(ll_3, (batch_size,1,1,1))
        y_l_3 = np.reshape(ll_3, (batch_size,1))

        
        # Perform mixup on both images and labels by combining a pair of images/labels
        # images_mixup_range_doppler = range_doppler_training_data_shuffled_1 * x_l + range_doppler_training_data_shuffled_2 * x_l_2 +\
        # range_doppler_training_data_shuffled_3 * x_l_3 + range_doppler_training_data_shuffled_4 * (1 - x_l - x_l_2 - x_l_3)
        # labels_mixup = labels_shuffled_1 * y_l + labels_shuffled_2 * y_l_2  + labels_shuffled_3 * y_l_3 + labels_shuffled_4 * (1 - y_l - y_l_2 - y_l_3)
        
        images_mixup_range_doppler = range_doppler_training_data_shuffled_1 * x_l + range_doppler_training_data_shuffled_2 * (1 - x_l)
        labels_mixup = labels_shuffled_1 * y_l + labels_shuffled_2 * (1 - y_l)

        concat_images_range_doppler[ii*batch_size:(ii+1)*batch_size,:,:,:] = images_mixup_range_doppler
        concat_label[ii*batch_size:(ii+1)*batch_size,:] = labels_mixup

      concat_images_range_doppler[repeat_of_mixup*batch_size:,:,:,:] = range_doppler_training_data
      concat_label[repeat_of_mixup*batch_size:,:] = labels
    return (concat_images_range_doppler, concat_label)
def vertical_mixup_augmentation(range_doppler_training_data, labels, repeat_of_mixup, alpha, beta):
    batch_size = range_doppler_training_data.shape[0]
    concat_images_range_doppler = np.zeros((batch_size*(repeat_of_mixup+1),range_doppler_training_data.shape[1],range_doppler_training_data.shape[2],range_doppler_training_data.shape[3]))
    concat_label = np.zeros((batch_size*(repeat_of_mixup+1),labels.shape[1]))

    # integer_training_labels = np.argmax(labels, axis=1)
    # np.where(integer_training_labels == 0)

    if repeat_of_mixup == 0:
      concat_images_range_doppler = range_doppler_training_data
      concat_label = labels      
    else:
      for ii in range(repeat_of_mixup):
        # shuffle train dataset
        shuffle_indx_1 = random.sample(range(0, range_doppler_training_data.shape[0]), range_doppler_training_data.shape[0]) # split validation data
        range_doppler_training_data_shuffled_1 = range_doppler_training_data[shuffle_indx_1,:,:,:]
        labels_shuffled_1 = labels[shuffle_indx_1,:]

        shuffle_indx_2 = random.sample(range(0, range_doppler_training_data.shape[0]), range_doppler_training_data.shape[0]) # split validation data
        range_doppler_training_data_shuffled_2 = range_doppler_training_data[shuffle_indx_2,:,:,:]
        labels_shuffled_2 = labels[shuffle_indx_2,:]

        images_mixup_range_doppler = np.zeros((range_doppler_training_data.shape))

        gaussian_mean = 0.2
        gaussian_std = 0.02
        ll_1 = np.random.normal(gaussian_mean, gaussian_std, (batch_size,1))
        vertical_resolution = range_doppler_training_data_shuffled_1.shape[2]
        index_number_1 = int(vertical_resolution*ll_1[0])
        index_number_2 = vertical_resolution - index_number_1
        shuffle_indx_1 = random.sample(range(0, vertical_resolution), index_number_1) # split validation data
        shuffle_indx_2 = np.delete(range(0, vertical_resolution), shuffle_indx_1) # split training data

        images_mixup_range_doppler[:,:,shuffle_indx_1,0] = range_doppler_training_data_shuffled_1[:,:,shuffle_indx_1,0]
        images_mixup_range_doppler[:,:,shuffle_indx_2,0] = range_doppler_training_data_shuffled_2[:,:,shuffle_indx_2,0]
        
        # images_mixup_range_doppler = range_doppler_training_data_shuffled_1 * x_l + range_doppler_training_data_shuffled_2 * (1 - x_l)

        labels_mixup = labels_shuffled_1 * ll_1 + labels_shuffled_2 * (1 - ll_1)

        concat_images_range_doppler[ii*batch_size:(ii+1)*batch_size,:,:,:] = images_mixup_range_doppler
        concat_label[ii*batch_size:(ii+1)*batch_size,:] = labels_mixup

      concat_images_range_doppler[repeat_of_mixup*batch_size:,:,:,:] = range_doppler_training_data
      concat_label[repeat_of_mixup*batch_size:,:] = labels
    return (concat_images_range_doppler, concat_label)
def split_and_augmentation_of_training(range_doppler_concat_shuffle_train,range_doppler_concat_label_shuffle_train,\
                                       repeat_of_mixup, augmentation_enable):
  # ---------------- Parameters ----------------
  # to split train and validation with same distribution
  alpha = 0.1
  beta = 5
  dummy_label = np.zeros((range_doppler_concat_shuffle_train.shape[0],1))
  for randomlist_for_train_indx, randomlist_for_validation_indx in kfold.split(range_doppler_concat_shuffle_train,dummy_label):   
    randomlist_for_validation_indx
  # size_of_validation = int(0.2*range_doppler_concat_shuffle_train.shape[0])
  # # Split validation
  # randomlist_for_validation_indx = random.sample(range(0, range_doppler_concat_shuffle_train.shape[0]), size_of_validation) # split validation data
  # randomlist_for_train_indx = np.delete(range(0, range_doppler_concat_shuffle_train.shape[0]), randomlist_for_validation_indx) # split training data
  # get validation data
  spectrogram_validation_labels = range_doppler_concat_label_shuffle_train[randomlist_for_validation_indx,:]
  range_doppler_validation_data = range_doppler_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  # get training data
  spectrogram_training_labels = spectrogram_concat_label_shuffle_train[randomlist_for_train_indx,:]
  range_doppler_training_data = range_doppler_concat_shuffle_train[randomlist_for_train_indx,:,:,:]

  # (range_doppler_augmented_image,spectrograms_label)=\
  #  mixup_augmentation(range_doppler_training_data, spectrogram_training_labels, repeat_of_mixup, alpha, beta)

  (range_doppler_augmented_image,spectrograms_label)=\
   vertical_mixup_augmentation(range_doppler_training_data, spectrogram_training_labels, repeat_of_mixup, alpha, beta)

  return (range_doppler_augmented_image,spectrograms_label,\
     range_doppler_validation_data, spectrogram_validation_labels)



def normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable):
  # ---------------- Normalize Inputs ----------------
  if normalize_inputs_enable == True:
    layer = Normalization(axis=None)
    layer.adapt(range_doppler_concat_shuffle)
    range_doppler_concat_shuffle = layer(range_doppler_concat_shuffle)
  else:
    range_doppler_concat_shuffle = range_doppler_concat_shuffle
  return(range_doppler_concat_shuffle)


In [ ]:
range_doppler_concat_shuffle_new = np.zeros((range_doppler_concat_shuffle.shape[0],216,216,1))
for ii in range(range_doppler_concat_shuffle.shape[0]):
  aa = range_doppler_concat_shuffle[ii,:,:,0]
  range_doppler_concat_shuffle_new[ii,:,:,0] = cv2.resize(aa, (216, 216),interpolation = cv2.INTER_CUBIC)
range_doppler_concat_shuffle = range_doppler_concat_shuffle_new
del aa
del range_doppler_concat_shuffle_new

In [ ]:
normalize_inputs_enable = 1
range_doppler_concat_shuffle = normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable)
# spectrogram_concat_shuffle = normalize_inputs(spectrogram_concat_shuffle, normalize_inputs_enable)
# third_spectrogram_concat_shuffle = normalize_inputs(third_spectrogram_concat_shuffle, normalize_inputs_enable)
range_doppler_concat_shuffle = np.float32(range_doppler_concat_shuffle)
# spectrogram_concat_shuffle = np.float32(spectrogram_concat_shuffle)
# third_spectrogram_concat_shuffle = np.float32(third_spectrogram_concat_shuffle)


In [ ]:
n_features = range_doppler_concat_shuffle.shape[1]
n_steps = range_doppler_concat_shuffle.shape[2]
range_doppler_concat_shuffle = np.transpose(range_doppler_concat_shuffle, axes = (0,2,1,3)) 
# spectrogram_concat_shuffle = np.transpose(spectrogram_concat_shuffle, axes = (0,2,1,3)) 
# third_spectrogram_concat_shuffle = np.transpose(third_spectrogram_concat_shuffle, axes = (0,2,1,3)) 

In [ ]:
# integer_training_labels = np.argmax(spectrogram_concat_label_shuffle_train, axis=1)
# np.where(integer_training_labels == 1)

In [ ]:
from keras.layers import *
from keras.models import *
from keras import backend as K

class attention(Layer):
    
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(attention,self).__init__()
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        # a = K.sigmoid(e)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [ ]:
t = time.time()
# ---------- Parameters ----------------
augmentation_enable = True
normalize_inputs_enable = True
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
test_accuracy_per_run = []
f1_score_per_run = []
epoch_number = 100
batch_size = 32
dense_size = 128
dropout_prob_dense = 0.4
repeat_of_mixup = 0
number_of_repeat = 5
lstm_dropout_rate = 0.4
unit_number_of_lstm = 64
dense_unit_of_lstm_function = 256
decoder_dense = 1024
for repeat_run_number in range(number_of_repeat):
  test_accuracy_per_fold = []
  f1_score_per_fold = []
  for train, test in kfold.split(range_doppler_concat_shuffle,range_doppler_concat_label_shuffle):   
    gc.collect()
    K.clear_session()

    randomlist_for_test_indx = test
    randomlist_for_train_indx = train
    # test data
    spectrogram_concat_label_shuffle_test = spectrogram_concat_label_shuffle[randomlist_for_test_indx,:]
    range_doppler_concat_shuffle_test = range_doppler_concat_shuffle[randomlist_for_test_indx,:,:,:]
    #train data
    spectrogram_concat_label_shuffle_train = spectrogram_concat_label_shuffle[randomlist_for_train_indx,:]
    range_doppler_concat_shuffle_train = range_doppler_concat_shuffle[randomlist_for_train_indx,:,:,:]
    # ---------------- Split labels to equal them during augmentation for Validation ----------------
    (range_doppler_augmented_image,spectrogram_concat_label_shuffle_concat,\
     validation_range_doppler, spectrogram_validation_labels)  =\
      split_and_augmentation_of_training(range_doppler_concat_shuffle_train,\
                                         spectrogram_concat_label_shuffle_train,\
                                         repeat_of_mixup, augmentation_enable)


    def lstm_encoder_network_1(input_shape):
      input = Input(shape=input_shape)
      x = Bidirectional(LSTM(unit_number_of_lstm, return_sequences=True, dropout = 0.))(input)       
      x = tf.transpose(x, perm=[0,2,1])
      x = attention(return_sequences=True)(x)
      x = tf.transpose(x, perm=[0,2,1])        
      x = GlobalAveragePooling1D()(x)

      return Model(input, x)

    # def lstm_encoder_network_1_reversed(input_shape):
    #   input = Input(shape=input_shape)
    #   x = tf.image.flip_up_down(input)
    #   x = (LSTM(unit_number_of_lstm, return_sequences=True, dropout = 0.))(x)       
    #   x = tf.transpose(x, perm=[0,2,1])
    #   x = attention(return_sequences=True)(x)
    #   x = tf.transpose(x, perm=[0,2,1])        
    #   x = GlobalAveragePooling1D()(x)

    #   return Model(input, x)


    def lstm_encoder_network_2(input_shape):
      input = Input(shape=input_shape)
      x = Bidirectional(GRU(unit_number_of_lstm, return_sequences=True, dropout = 0.))(input)
      x = tf.transpose(x, perm=[0,2,1])
      x = attention(return_sequences=True)(x)
      x = tf.transpose(x, perm=[0,2,1])        
      x = GlobalAveragePooling1D()(x)
      return Model(input, x)
    
    # def lstm_encoder_network_2_reversed(input_shape):
    #   input = Input(shape=input_shape)
    #   x = tf.image.flip_up_down(input)
    #   x = (GRU(unit_number_of_lstm, return_sequences=True, dropout = 0.))(x)
    #   x = tf.transpose(x, perm=[0,2,1])
    #   x = attention(return_sequences=True)(x)
    #   x = tf.transpose(x, perm=[0,2,1])        
    #   x = GlobalAveragePooling1D()(x)
    #   return Model(input, x)

    # def lstm_encoder_network_3(input_shape):
    #   input = Input(shape=input_shape)
    #   conv1 = Conv1D(filters=64, kernel_size=3, padding="same")(input)
    #   conv1 = BatchNormalization()(conv1)
    #   conv1 = ReLU()(conv1)
    #   conv2 = Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    #   conv2 = BatchNormalization()(conv2)
    #   conv2 = ReLU()(conv2)

    #   conv3 = Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    #   conv3 = BatchNormalization()(conv3)
    #   x = ReLU()(conv3)
    #   x = tf.transpose(x, perm=[0,2,1])
    #   x = attention(return_sequences=True)(x)
    #   x = tf.transpose(x, perm=[0,2,1])        
    #   x = GlobalAveragePooling1D()(x)

    #   return Model(input, x)

    def decoder_for_concat(input_shape):
      input = Input(shape=input_shape)
      # x = Dense(1024)(input)
      # x = BatchNormalization()(x)
      # x = Activation('LeakyReLU')(x)
      # x = Dropout(0.5)(x)
      x = Dense(64)(input)
      x = BatchNormalization()(x)
      x = Activation('LeakyReLU')(x)
      x = Dropout(0.45)(x)
      x = Dense(11, activation="softmax")(x)
      return Model(input, x)

    # def build_model(
    #   input_shape,
    #   head_size,
    #   num_heads,
    #   ff_dim,
    #   num_transformer_blocks,
    #   mlp_units,
    #   dropout=0,
    #   mlp_dropout=0,
    # ):
    #   inputs = Input(shape=input_shape)
    #   x = inputs
    #   for _ in range(num_transformer_blocks):
    #       x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    #   x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    #   for dim in mlp_units:
    #       x = layers.Dense(dim, activation="relu")(x)
    #       outputs = layers.Dropout(mlp_dropout)(x)
    #   # outputs = layers.Dense(11, activation="softmax")(x)
    #   return Model(inputs, outputs)

    # input_shape = range_doppler_concat_shuffle.shape[1:3]

    # model_t = build_model(
    #     input_shape,
    #     head_size=256,
    #     num_heads=4,
    #     ff_dim=4,
    #     num_transformer_blocks=4,
    #     mlp_units=[128],
    #     mlp_dropout=0.4,
    #     dropout=0.25,
    # )

    # lstm_input_t  = Input(shape=input_shape)
    # processed_lstm_1_t  = model_t(lstm_input_t)

    input_shape = range_doppler_concat_shuffle.shape[1:3]

    base_network_lstm_1 = lstm_encoder_network_1(input_shape)
    lstm_input_1  = Input(shape=input_shape)
    processed_lstm_1  = base_network_lstm_1(lstm_input_1)


    base_network_lstm_2 = lstm_encoder_network_2(input_shape)
    lstm_input_2  = Input(shape=input_shape)
    processed_lstm_2  = base_network_lstm_2(lstm_input_1)

    # base_network_lstm_1_reversed = lstm_encoder_network_1_reversed(input_shape)
    # lstm_input_1_reversed  = Input(shape=input_shape)
    # processed_lstm_1_reversed  = base_network_lstm_1(lstm_input_1_reversed)


    # base_network_lstm_2_reversed = lstm_encoder_network_2_reversed(input_shape)
    # lstm_input_2_reversed  = Input(shape=input_shape)
    # processed_lstm_2_reversed  = base_network_lstm_2(lstm_input_2_reversed)


    # base_network_lstm_3 = lstm_encoder_network_3(input_shape)
    # lstm_input_3  = Input(shape=input_shape)
    # processed_lstm_3  = base_network_lstm_3(lstm_input_1)

    concat_layer = Concatenate()([processed_lstm_1, processed_lstm_2])



    base_decoder_network = decoder_for_concat((concat_layer.shape[1]))
    out = base_decoder_network(concat_layer)

    model = Model(inputs=[lstm_input_1], outputs=[out])

    print(base_network_lstm_1.summary()) 
    print(base_network_lstm_2.summary()) 
    # print(model_t.summary()) 
    print(base_decoder_network.summary())
    # ---------------- Compile and Fit ----------------
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    earlyStopping = EarlyStopping(monitor='val_loss', patience=30, verbose=0,restore_best_weights=True, mode='min')
    # earlyStopping = EarlyStopping(monitor='val_accuracy', patience=15, verbose=0,restore_best_weights=True, mode='max')
    history = model.fit((range_doppler_augmented_image),(spectrogram_concat_label_shuffle_concat),
                    epochs=epoch_number,
                    batch_size=batch_size,
                    shuffle = True,
                    callbacks=[earlyStopping],
                    validation_data = ((validation_range_doppler) , (spectrogram_validation_labels)))
    tf.keras.models.load_model
    test_loss, test_accuracy  = model.evaluate([range_doppler_concat_shuffle_test],\
                                               [spectrogram_concat_label_shuffle_test],
                  batch_size=batch_size)
    # ---------------- Get Test Results ----------------
    # y_test_predicted = model.predict((range_doppler_concat_shuffle_test,spectrogram_concat_shuffle_test,third_spectrogram_concat_shuffle_test), batch_size=batch_size)
    # # ----- Binarize y_test_predicted values -----
    # y_test_predicted_binary = np.zeros(y_test_predicted.shape[0])
    # for ii in range(y_test_predicted.shape[0]):
    #   if argmax(y_test_predicted[ii,:]) == 0:
    #     y_test_predicted_binary[ii] = 0
    #   elif argmax(y_test_predicted[ii,:]) == 1:
    #     y_test_predicted_binary[ii] = 1
    #   else:
    #     y_test_predicted_binary[ii] = 2
    # range_doppler_concat_label_shuffle_test1 = range_doppler_concat_label_shuffle[randomlist_for_test_indx,:]
    # test_precision, test_recall, test_f1_score, support = precision_recall_fscore_support(range_doppler_concat_label_shuffle_test1, y_test_predicted_binary, average='macro')
    # ----------------------------------------------------------------------------------------------------------
    test_accuracy_per_fold.append(test_accuracy)
    # f1_score_per_fold.append(test_f1_score)
    del model
  test_accuracy_per_run.append(sum(test_accuracy_per_fold)/num_folds)
  # f1_score_per_run.append(sum(f1_score_per_fold)/num_folds)
  print(test_accuracy_per_run)
  # print(f1_score_per_run)

print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)},\
 max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))},\
 min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, \
 std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}.')
elapsed = time.time() - t
print(f'Time elapsed through all process: {"{:.2f}".format(elapsed)}, sec')

In [ ]:
print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)},\
 max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))},')
print(f'Min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, \
 std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}.')
print(f'Time elapsed through all process: {"{:.2f}".format(elapsed)} sec')

Mean test accuracy is 0.858, max test accuracy is 0.863,
Min test accuracy is 0.855,  std of test accuracy is 0.003.
Time elapsed through all process: 1799.72 sec
